In [6]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [8]:
from llama_index.llms.openai import OpenAI
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

In [ ]:
llm = HuggingFaceInferenceAPI(
    model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    temperature=0.1,
    token=os.environ["HF_TOKEN"],
    provider="nebius",
)
response = llm.complete("William Shakespeare is ", max_tokens=100)

In [14]:
print(response)

William Shakespeare (1564–1616) was an English playwright, poet, and actor, widely regarded as one of the greatest writers in the English language and the world's preeminent dramatist. He wrote 39 plays, 154 sonnets, and two long narrative poems. His works include famous tragedies such as *Hamlet*, *Macbeth*, *Romeo and Juliet*, and *Othello*; comedies like *A


In [15]:
handle = llm.stream_complete("William Shakespeare is ", max_tokens=20)
for token in handle:
    print(token.delta, end="", flush=True)

William Shakespeare (1564–1616) was an English playwright, poet,

In [16]:
from llama_index.core.tools import FunctionTool

In [18]:
def generate_song(name: str, artist: str):
    """Generates a song with provided name and artist."""
    return {"name": name, "artist": artist}


tool = FunctionTool.from_defaults(fn=generate_song)

In [22]:
tool.metadata

ToolMetadata(description='generate_song(name: str, artist: str)\nGenerates a song with provided name and artist.', name='generate_song', fn_schema=<class 'llama_index.core.tools.utils.generate_song'>, return_direct=False)

In [31]:
from llama_index.llms.openai_like import OpenAILike
import os

# Configure specifically for Nebius
llm = OpenAILike(
    model="Qwen/Qwen3-30B-A3B-Instruct-2507",  # Use the exact model ID Nebius supports
    api_key=os.environ["NEBIUS_API_KEY"],  # Ensure this matches your env var
    api_base=os.environ["NEBIUS_BASE_URL"],
    temperature=0.1,
    max_tokens=4096,
    context_window=262000,
    is_chat_model=True,
    is_function_calling_model=True,
)

In [32]:
response = llm.predict_and_call([tool], "Pick a random song for me")
print(str(response))

{'name': 'Random Song', 'artist': 'Unknown Artist'}


# try building agents

In [30]:
from dotenv import load_dotenv
import os
from dataclasses import asdict

load_dotenv()

True

In [2]:
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.callbacks import TokenCountingHandler
import tiktoken

In [3]:
from transformers import AutoTokenizer

In [4]:
model = "Qwen/Qwen3-30B-A3B-Instruct-2507"
tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
import logging
import sys

# Configure logging to print everything to stdout
logging.basicConfig(stream=sys.stdout, level=logging.WARN)

# Enable DEBUG logs specifically for the HTTP client used by LlamaIndex/OpenAI
logging.getLogger("httpx").setLevel(logging.INFO)
logging.getLogger("httpcore").setLevel(logging.INFO)
logging.getLogger("openai").setLevel(logging.INFO)

In [5]:
from llama_index.core.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.core import Settings

# 1. Initialize the debug handler
# llama_debug = LlamaDebugHandler(print_trace_on_end=True)
token_counter = TokenCountingHandler(
    tokenizer=tokenizer.encode  # generic fallback
)

callback_manager = CallbackManager([token_counter])

# 2. Apply it globally
Settings.callback_manager = callback_manager

In [6]:
llm = OpenAILike(
    model=model,
    api_key=os.environ["NEBIUS_API_KEY"],
    api_base=os.environ["NEBIUS_BASE_URL"],
    temperature=0.1,
    max_tokens=4000,
    context_window=262000,
    is_chat_model=True,
    is_function_calling_model=True,
)

In [7]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b


def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b


def subtract(a: float, b: float) -> float:
    """Subtract one number from the other and return the difference"""
    return a - b

In [8]:
workflow = FunctionAgent(
    tools=[multiply, add, subtract],
    llm=llm,
    system_prompt="You are an agent that can perform basic mathematical operations using tools.",
    verbose=False,
    timeout=30,
    max_tokens=2000,
    streaming=False,
)

In [9]:
# 1. clear previous history
token_counter.reset_counts()

# 2. Execute
response = await workflow.run(
    user_msg="What is the value of the expression $(1 + 2)*(3-4)+5$? Show your steps and reasoning."
)

# 3. Stats
print(
    f"Token Usage: {token_counter.total_llm_token_count} (In: {token_counter.prompt_llm_token_count}, Out: {token_counter.completion_llm_token_count})"
)

# 4. Trace
print("\n--- Execution Trace ---")
for step in response.tool_calls:
    print(f"🔧 Called Tool: {step.tool_name}")
    print(f"   ├── Args:   {step.tool_kwargs}")
    print(f"   └── Result: {str(step.tool_output.raw_output)[:100]}")


Token Usage: 1587 (In: 1173, Out: 414)

--- Execution Trace ---
🔧 Called Tool: add
   ├── Args:   {'a': 1, 'b': 2}
   └── Result: 3
🔧 Called Tool: subtract
   ├── Args:   {'a': 3, 'b': 4}
   └── Result: -1
🔧 Called Tool: multiply
   ├── Args:   {'a': 3, 'b': -1}
   └── Result: -3
🔧 Called Tool: add
   ├── Args:   {'a': -3, 'b': 5}
   └── Result: 2


In [10]:
print(response)

The steps and calculations are as follows:

1. $1 + 2 = 3$ (using the `add` function).
2. $3 - 4 = -1$ (using the `subtract` function).
3. $3 \times (-1) = -3$ (using the `multiply` function).
4. $-3 + 5 = 2$ (using the `add` function).

Thus, the value of the expression $(1 + 2) \times (3 - 4) + 5$ is $2$.


## using existing tools

In [9]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec

In [44]:
ddgs = DuckDuckGoSearchToolSpec()
ddgs.to_tool_list()
agent = FunctionAgent(
    name="Web Search Agent",
    tools=ddgs.to_tool_list(),
    llm=llm,
    system_prompt="You are a helpful agent whose job is to understand the user's query and search the internet given the tools provided to you. Today's date is Wednesday, November 26th 2025. The user is in India. The language is English",
)

In [40]:
for tool in ddgs.to_tool_list():
    print(tool.metadata.name)
    print()

duckduckgo_instant_search

duckduckgo_full_search



In [46]:
token_counter.reset_counts()
response = await agent.run(
    "Did Gautam Gambhir resign? Do a full search on the internet."
)
print(response)
print(
    f"Token Usage: {token_counter.total_llm_token_count} (In: {token_counter.prompt_llm_token_count}, Out: {token_counter.completion_llm_token_count})"
)
print("\n--- Execution Trace ---")
for step in response.tool_calls:
    print(f"🔧 Called Tool: {step.tool_name}")
    print(f"   ├── Args:   {step.tool_kwargs}")
    print(f"   └── Result: {str(step.tool_output.raw_output)[:100]}")

**********
Trace: chat
    |_CBEventType.LLM -> 0.0 seconds
**********
INFO:httpx:HTTP Request: POST https://api.tokenfactory.nebius.com/v1/chat/completions "HTTP/1.1 200 OK"
**********
Trace: chat
    |_CBEventType.LLM -> 0.0 seconds
**********
INFO:httpx:HTTP Request: POST https://api.tokenfactory.nebius.com/v1/chat/completions "HTTP/1.1 200 OK"
I cannot access the full content of the search results due to a ratelimit error. However, based on the available information, Gautam Gambhir has not officially resigned from his roles. He remains active in cricket administration and media, including his role with the Delhi Capitals in the IPL and his work as a commentator. For the most accurate and up-to-date information, I recommend checking reliable news sources or official announcements.
Token Usage: 376 (In: 287, Out: 89)

--- Execution Trace ---
🔧 Called Tool: duckduckgo_full_search
   ├── Args:   {'query': 'Gautam Gambhir resign', 'region': 'in-en', 'max_results': 10}
   └── Result: htt

In [43]:
response.tool_calls

[ToolCallResult(tool_name='duckduckgo_instant_search', tool_kwargs={'query': 'Gautam Gambhir resign'}, tool_id='chatcmpl-tool-b6fce66cb8614b3eb09eceda80baccaf', tool_output=ToolOutput(blocks=[], tool_name='duckduckgo_instant_search', raw_input={'args': (), 'kwargs': {'query': 'Gautam Gambhir resign'}}, raw_output=[], is_error=False), return_direct=False)]

## context

In [11]:
from llama_index.core.workflow import Context
from llama_index.core.agent import ReActAgent


agent = ReActAgent(name="Simple agent", llm=llm)

In [12]:
ctx = Context(agent)

In [15]:
async def _r(agent, *args, method: str = "run", token_counter=token_counter, **kwargs):
    """
    Runner helper that handles token counting, execution, and tracing.

    Args:
        agent: The agent workflow or engine.
        *args: Positional arguments for the agent's method (e.g., the query string).
        method (str): The method to call on the agent (default: "run").
        token_counter: The counter to reset/read (defaults to global variable).
        **kwargs: Keyword arguments passed directly to the agent's method.
    """
    # 1. Reset Token Counts
    if token_counter:
        token_counter.reset_counts()

    # 2. Execute the Agent Method
    # This dynamically calls agent.run(*args, **kwargs) or agent.chat(...)
    response = await getattr(agent, method)(*args, **kwargs)

    # 3. Print Final Response
    print(f"🤖 Response: {str(response)}\n")

    # 4. Print Token Statistics
    if token_counter:
        print(
            f"📊 Token Usage: {token_counter.total_llm_token_count} "
            f"(In: {token_counter.prompt_llm_token_count}, Out: {token_counter.completion_llm_token_count})"
        )

    # 5. Print Execution Trace (Tool Calls)
    # We check for 'tool_calls' (Workflow) or fallback to 'sources' (ReAct)
    steps = getattr(response, "tool_calls", []) or getattr(response, "sources", [])

    if steps:
        print("\n--- 🛠️ Execution Trace ---")
        for step in steps:
            # Safe attribute access to handle different agent types
            tool_name = getattr(step, "tool_name", "Unknown")
            tool_args = getattr(step, "tool_kwargs", getattr(step, "tool_input", {}))

            # Dig for the result: FunctionAgent nests it in tool_output.raw_output
            output_container = getattr(step, "tool_output", None)
            if output_container:
                raw_result = getattr(
                    output_container, "raw_output", str(output_container)
                )
            else:
                # Fallback for ReActAgent which puts it in raw_output directly
                raw_result = getattr(step, "raw_output", "No Output")

            print(f"🔧 Called Tool: {tool_name}")
            print(f"   ├── Args:   {tool_args}")
            print(f"   └── Result: {str(raw_result)[:150]}...")
            print("")

    return response


In [19]:
_ = await _r(agent, "what is your name?", ctx=ctx)

🤖 Response: My name is y.

📊 Token Usage: 752 (In: 720, Out: 32)


In [20]:
from llama_index.core.agent.workflow import AgentWorkflow

In [21]:
async def set_name(ctx: Context, name: str) -> str:
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["name"] = name

    return f"Name set to {name}"

In [22]:
workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

In [23]:
ctx = Context(workflow)

In [25]:
_ = await _r(workflow, "What's my name?", ctx=ctx)

🤖 Response: I don't know your name yet. Would you like to set it?

📊 Token Usage: 53 (In: 36, Out: 17)


In [26]:
_ = await _r(workflow, "My name is X.", ctx=ctx)

🤖 Response: Your name is now set to X. How can I assist you today?

📊 Token Usage: 193 (In: 173, Out: 20)

--- 🛠️ Execution Trace ---
🔧 Called Tool: set_name
   ├── Args:   {'name': 'X'}
   └── Result: Name set to X...



In [27]:
state = await ctx.store.get("state")

In [28]:
print("Name as stored in state: ", state["name"])

Name as stored in state:  X


## streaming outputs

In [31]:
from llama_index.tools.tavily_research import TavilyToolSpec

In [32]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))

In [33]:
workflow = FunctionAgent(
    tools=tavily_tool.to_tool_list(),
    llm=llm,
    system_prompt="You're a helpful assistant that can search the web for information.",
)

In [39]:
from llama_index.core.agent.workflow import (
    AgentInput,
    AgentOutput,
    ToolCall,
    ToolCallResult,
    AgentStream,
)

In [43]:
handler = workflow.run(user_msg="What's the weather like in Chennai?")

# handle streaming output
async for event in handler.stream_events():
    # if isinstance(event, AgentStream):
    #     print(event.delta, end="", flush=True)
    if isinstance(event, AgentInput):
        print("Agent input: ", event.input)  # the current input messages
        print("Agent name:", event.current_agent_name)  # the current agent name
    # elif isinstance(event, AgentOutput):
    #     print("=" * 10)
    #     print("Agent output: ", event.response)  # the current full response
    #     print("Tool calls made: ", event.tool_calls)  # the selected tool calls, if any
    #     print("Raw LLM response: ", event.raw)  # the raw llm api response
    #     print("=" * 10)
    # elif isinstance(event, ToolCallResult):
    #     print("=" * 10)
    #     print("Tool called: ", event.tool_name)  # the tool name
    #     print("Arguments to the tool: ", event.tool_kwargs)  # the tool kwargs
    #     print("Tool output: ", event.tool_output)  # the tool output
    #     print("=" * 10)

Agent input:  [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="You're a helpful assistant that can search the web for information.")]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="What's the weather like in Chennai?")])]
Agent name: Agent
Agent input:  [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="You're a helpful assistant that can search the web for information.")]), ChatMessage(role=<MessageRole.USER: 'user'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text="What's the weather like in Chennai?")]), ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': [ChoiceDeltaToolCall(index=0, id='chatcmpl-tool-3d4a0418c4084df699268b650504211c', function=ChoiceDeltaToolCallFunction(arguments='{"query": "current weather in Chennai", "max_results": 5}

## human in the loop

In [45]:
from llama_index.core.workflow import (
    InputRequiredEvent,
    HumanResponseEvent,
)

In [ ]:
async def dangerous_task(ctx: Context) -> str:
    """A dangerous task that requires human confirmation."""

    # emit a waiter event (InputRequiredEvent here)
    # and wait until we see a HumanResponseEvent
    question = "Are you sure you want to proceed? "
    response = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id=question,
        waiter_event=InputRequiredEvent(
            prefix=question,
            user_name="Laurie",
        ),
        requirements={"user_name": "Laurie"},
    )

    # act on the input from the event
    if response.response.strip().lower() == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted."

In [46]:
workflow = FunctionAgent(
    tools=[dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant that can perform dangerous tasks.",
)

In [ ]:
handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
    if isinstance(event, InputRequiredEvent):
        # capture keyboard input
        response = input(event.prefix)
        # send our response back
        handler.ctx.send_event(
            HumanResponseEvent(
                response=response,
                user_name=event.user_name,
            )
        )

response = await handler
print(str(response))

In [51]:
from llama_index.core.workflow import (
    Context,
    Event,
)
from llama_index.core.agent import FunctionAgent


# Define events explicitly
class InputRequiredEvent(Event):
    prefix: str
    user_name: str


class HumanResponseEvent(Event):
    response: str
    user_name: str


# --- THE FIX: Hide 'ctx' from the LLM ---
# LlamaIndex tools inspect type hints. If you type hint 'Context',
# the agent might try to generate an argument for it if not configured correctly.
# But standard FunctionAgent handles 'ctx: Context' injection automatically
# IF it's the first argument.


async def dangerous_task(ctx: Context) -> str:
    """
    A dangerous task that requires human confirmation.
    """
    # emit a waiter event
    question = "Are you sure you want to proceed? (yes/no)"

    # This pauses execution and waits for the event
    # The LLM does NOT see this part; it just sees the tool called.
    response_event = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id="human_confirmation",  # Fixed ID ensures we find it
        waiter_event=InputRequiredEvent(
            prefix=question,
            user_name="User",
        ),
    )

    # act on the input
    # access the .response field from your Event class
    ans = response_event.response.strip().lower()

    if ans == "yes":
        return "Dangerous task completed successfully."
    else:
        return "Dangerous task aborted by user."


# 2. Initialize Agent with streaming=False
workflow = FunctionAgent(
    tools=[dangerous_task],
    llm=llm,
    system_prompt="You are a helpful assistant.",
    verbose=True,
    streaming=False,  # <--- CRITICAL for stability with HITL
)

# 3. Run the Workflow manually
# We use the lower-level run() to handle the async iterator manually
handler = workflow.run(user_msg="I want to proceed with the dangerous task.")

async for event in handler.stream_events():
    # 4. Intercept the specific event we defined
    if isinstance(event, InputRequiredEvent):
        print(f"\n✋ INTERRUPT: {event.prefix}")

        # Get input from Jupyter/Console
        user_input = input("Your Answer: ")

        # Send it back to the workflow
        handler.ctx.send_event(
            HumanResponseEvent(
                response=user_input,
                user_name=event.user_name,
            )
        )

# 5. Get Final Result
final_response = await handler
print(f"\n🤖 Final Response: {str(final_response)}")



✋ INTERRUPT: Are you sure you want to proceed? (yes/no)

🤖 Final Response: The dangerous task has been completed successfully. Let me know if you need further assistance!


# multi agent workflows

## linear swarm

In [56]:
from tavily import AsyncTavilyClient
from llama_index.core.workflow import Context

tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))


async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    return str(await client.search(query))


async def record_notes(ctx: Context, notes: str, notes_title: str) -> str:
    """Useful for recording notes on a given topic. Your input should be notes with a title to save the notes under."""
    async with ctx.store.edit_state() as ctx_state:
        if "research_notes" not in ctx_state["state"]:
            ctx_state["state"]["research_notes"] = {}
        ctx_state["state"]["research_notes"][notes_title] = notes
    return "Notes recorded."


async def write_report(ctx: Context, report_content: str) -> str:
    """Useful for writing a report on a given topic. Your input should be a markdown formatted report."""
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["report_content"] = report_content
    return "Report written."


async def review_report(ctx: Context, review: str) -> str:
    """Useful for reviewing a report and providing feedback. Your input should be a review of the report."""
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["review"] = review
    return "Report reviewed."

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = FunctionAgent(
    name="ResearchAgent",
    description="Useful for searching the web for information on a given topic and recording notes on the topic.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "Once notes are recorded and you are satisfied, you should hand off control to the WriteAgent to write a report on the topic. "
        "You should have at least some notes on a topic before handing off control to the WriteAgent."
    ),
    llm=llm,
    tools=[search_web, record_notes],
    can_handoff_to=["WriteAgent"],
)

write_agent = FunctionAgent(
    name="WriteAgent",
    description="Useful for writing a report on a given topic.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm=llm,
    tools=[write_report],
    can_handoff_to=["ReviewAgent", "ResearchAgent"],
)

review_agent = FunctionAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review the write report and provide feedback. "
        "Your review should either approve the current report or request changes for the WriteAgent to implement. "
        "If you have feedback that requires changes, you should hand off control to the WriteAgent to implement the changes after submitting the review."
    ),
    llm=llm,
    tools=[review_report],
    can_handoff_to=["WriteAgent"],
)

In [54]:
from llama_index.core.agent.workflow import AgentWorkflow

agent_workflow = AgentWorkflow(
    agents=[research_agent, write_agent, review_agent],
    root_agent=research_agent.name,
    initial_state={
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

In [60]:
# 1. Ensure your token counter is accessible here (global variable)
# token_counter = ... (your existing counter)

handler = agent_workflow.run(
    user_msg=(
        "Write me a report on the history of the internet. "
        "Briefly describe the history of the internet, including the development of the internet, the development of the web, "
        "and the development of the internet in the 21st century."
    )
)
token_counter.reset_counts()
current_agent = None
total_in = 0
total_out = 0
in_rate = 0.1 / 1_000_000
out_rate = 0.3 / 1_000_000
async for event in handler.stream_events():
    flag = 0
    if (
        hasattr(event, "current_agent_name")
        and event.current_agent_name != current_agent
    ):
        current_agent = event.current_agent_name
        print(f"\n{'=' * 50}")
        print(f"🤖 Agent: {current_agent}")
        print(f"{'=' * 50}\n")
        flag = 1

    elif isinstance(event, AgentOutput):
        if event.response.content:
            print(f"📤 Output: {event.response.content}")
        if event.tool_calls:
            print(
                "🛠️  Planning to use tools:",
                [call.tool_name for call in event.tool_calls],
            )
        flag = 1
    elif isinstance(event, ToolCallResult):
        print(f"🔧 Tool Result ({event.tool_name}):")
        print(f"  Arguments: {event.tool_kwargs}")
        # Truncate long outputs for sanity
        out_str = str(
            event.tool_output.raw_output
            if hasattr(event.tool_output, "raw_output")
            else event.tool_output
        )
        print(f"  Output: {out_str[:200]}...")
        flag = 1
    elif isinstance(event, ToolCall):
        print(f"🔨 Calling Tool: {event.tool_name}")
        print(f"  With arguments: {event.tool_kwargs}")
        flag = 1

    if flag == 1:
        # --- NEW: Print Token Delta ---
        # We print usage on every event so you can see "what cost what"
        total = token_counter.total_llm_token_count
        in_t = token_counter.prompt_llm_token_count
        out_t = token_counter.completion_llm_token_count
        total_in += in_t
        total_out += out_t
        print(f"\nToken Usage: {total}")
        print(
            f"(In: {in_t} (${in_t * in_rate:0.4f}), Out: {out_t} (${out_t * out_rate:0.4f}))\n"
        )
        token_counter.reset_counts()
        # ------------------------------


print(f"{'=' * 50}\n")
print(f"\nToken Usage: {total_in + total_out}")
print(
    f"(In: {total_in} (${total_in * in_rate:0.4f}), Out: {total_out} (${total_out * out_rate:0.4f}))\n"
)


🤖 Agent: ResearchAgent


Token Usage: 0
(In: 0 ($0.0000), Out: 0 ($0.0000))

🛠️  Planning to use tools: ['search']

Token Usage: 3008
(In: 3005 ($0.0003), Out: 3 ($0.0000))

🔨 Calling Tool: search
  With arguments: {'query': 'history of the internet development internet web 21st century', 'max_results': 6}

Token Usage: 0
(In: 0 ($0.0000), Out: 0 ($0.0000))

🔧 Tool Result (search):
  Arguments: {'query': 'history of the internet development internet web 21st century', 'max_results': 6}
  Output: [Document(id_='560428c7-38b7-4a95-b8c1-b0ea2d1d7ac9', embedding=None, metadata={'url': 'https://www.uswitch.com/broadband/guides/broadband-history/'}, excluded_embed_metadata_keys=[], excluded_llm_met...

Token Usage: 0
(In: 0 ($0.0000), Out: 0 ($0.0000))

🛠️  Planning to use tools: ['extract']

Token Usage: 4381
(In: 4378 ($0.0004), Out: 3 ($0.0000))

🔨 Calling Tool: extract
  With arguments: {'urls': ['https://www.history.com/topics/inventions/history-of-the-internet', 'https://www.britannic

In [61]:
state = await handler.ctx.store.get("state")
print(state["report_content"])

# The History of the Internet: From ARPANET to Web3

## Introduction

The internet has become one of the most transformative inventions in human history, reshaping how we communicate, work, learn, and entertain ourselves. From its origins as a military research project to its current status as a global digital infrastructure, the internet’s evolution reflects decades of innovation, collaboration, and technological breakthroughs. This report explores the key milestones in the history of the internet, focusing on three major phases: the development of the internet itself, the creation of the World Wide Web, and the rapid advancements of the 21st century.

## 1. Early Development (1960s–1970s)

### The Birth of ARPANET

The story of the internet begins in the late 1960s with the creation of ARPANET (Advanced Research Projects Agency Network), a project funded by the U.S. Department of Defense’s Advanced Research Projects Agency (ARPA). In 1969, ARPANET connected four major U.S. universiti